In [3]:
# import the necessary packages
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [4]:
auth_tokens = []
f = open("academic_auth", "r")
for line in f:
    auth_tokens.append(line.strip())
f.close()

keyword=['myanmar']

In [5]:
print(auth_tokens)

['ZIB9CVIrCjQOiXXpCAPcYgWNb', 'lWaEV4w9FtCeyPaq3aGrzlKaDVrbwuvAgpIDXlpCd6ZF4041vw', '4738148390-eoQ7rW2XuyWsDb438k0hcRbIcO2usX1H5L5A72j', 'ih1Rx1aOuui9p3ZKSKer9tSnKX1JbrlTTf04KF69GVZ5o']


In [6]:
class TweetsListener(StreamListener):
  # tweet object listens for the tweets
    def __init__(self, csocket):
        self.client_socket = csocket
    def on_data(self, data):
        try:  
            msg = json.loads(data)
            print("new message")
            # if tweet is longer than 140 characters
            if "extended_tweet" in msg:
                # add at the end of each tweet "t_end"
                #tweet=str(msg['extended_tweet']['full_text']+"t_end").encode('utf-8')
                self.client_socket.send(data.encode())
                print(msg['extended_tweet']['full_text'])
            else:
                # add at the end of each tweet "t_end"
                #tweet=str(msg['text']+"t_end").encode('utf-8')
                self.client_socket.send(data.encode())
                print(msg['text'])
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True

In [7]:
def sendData(c_socket, keyword):
    print('start sending data from Twitter to socket')
    # authentication based on the credentials
    auth = OAuthHandler(auth_tokens[0], auth_tokens[1])
    auth.set_access_token(auth_tokens[2], auth_tokens[3])
    # start sending data from the Streaming API 
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track = keyword, languages=["en"])

In [ ]:
# server (local machine) creates listening socket
s = socket.socket()
host = "0.0.0.0"    
port = 5555
s.bind((host, port))
print('socket is ready')
# server (local machine) listens for connections
s.listen(4)
print('socket is listening')
# return the socket and the address on the other side of the connection (client side)
c_socket, addr = s.accept()
print("Received request from: " + str(addr))
# select here the keyword for the tweet data
sendData(c_socket, keyword)

socket is ready
socket is listening
Received request from: ('192.168.10.141', 52092)
start sending data from Twitter to socket
new message
RT @Myanmar_Now_Eng: In Taze, Sagaing, locals opposed to Myanmar’s military junta held a motorcycle rally this morning (May 26) and critici…
new message
Save Our Children!

#CrimesAgainstChildren
#WhatsHappeningInMyanmar
#May26Coup https://t.co/cu8JUQgLqE
new message
RT @Kyi_Sin_Thant: Mandalay, Myanmar, has brought back three-finger salutes and car horns. Protesters want to show their disapproval of the…
new message
Mandalay, Myanmar, has brought back three-finger salutes and car horns. Protesters want to show their disapproval of the coup and the junta without being killed. #May27Coup #AuspiceRefugees #WhatsHappeningInMyanmar https://t.co/Zk3vyG1Ud5
new message
Monks from Masoeyein monastery shared their prayers for the Myanmar people.

#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar https://t.co/tqFSAkP0U6
new message
@FrontierMM @paolapeaco

new message
RT @WMU_Myanmar: Little Support of Snowballpowers
for IDPs(who're displaced by war)in Myanmar.
Even in our darkest hours,@winmetawin  is th…
new message
The truth cannot be hidden
#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar https://t.co/hUIfAKeZ6e
new message
Buddhist monks in SaTaung, Sagaing, prayed for the abducted monks and country's leaders to be released immediately, as well as for the fallen martyrs and Myanmar's people. #May27Coup #AuspiceRefugees #WhatsHappeningInMyanmar https://t.co/EtlXXYmfKi
new message
RT @Myanmar_Now_Eng: Locals in Bagan, famous for its stunning landscape filled with pagodas, came out today (May 25) to protest against Mya…
new message
This is one of our success ✊

#WhatsHappeningInMyanmar 
#May26Coup https://t.co/QUndH9wfGG
new message
RT @cvdom2021: Buddhist monks around the world boycott the military dictatorship and show solidarity with democracy movement in Myanmar. Ma…
new message
from myanmar: “ they shoot at heads. but they do 

new message
RT @TotalPress: 🇲🇲 In light of the context in #Myanmar, and following a joint proposal by @Total &amp; @Chevron, shareholders of MGTC vote to s…
new message
Mandalay and Sagaing: Monks from Mogaung Monastery in Mandalay and SarTaung monastery in Sagaing, held the candlelight and prayed for Myanmar People to break free from dictatorship and at peace. #WhatsHappeningInMyanmar #May26Coup https://t.co/49xfGSdH8c
new message
@Total and @Chevron has announced they have suspended payments to Myanmar military terrorists. Sincerely grateful to  @Total and @Chevron for stopp aiding #MurdererMinAungHlaing. #WhatsHappeningInMyanmar  #AuspiceRefugees  #May27Coup https://t.co/Y2LV20Cda6
new message
RT @MayWongCNA: #Myanmar civilian government @NUGMyanmar planning &amp; investment ministry issues statement reiterating that all investment pe…
new message
RT @CSW_UK: #MYANMAR/#BURMA: 'The international sanctions introduced so far have been welcome, but states can and must do more to cut off

new message
Every night, residents of WunTho Tsp, Sagaing Division, hold a praying service ceremony in which they chant "24 Patthana" to restore peace to Myanmar. #May26Coup #AuspiceRefugees #WhatsHappeningInMyanmar https://t.co/BN3yiGuicV
new message
RT @LeongWaiKitCNA: #Japan plans to allow #Myanmar citizens whose visas expired there, to continue legally staying in country 

Separately,…
new message
The Metta Sutta and Morat Sutta prayers for immediate peace for all Myanmar people and the release of all detainees were recited by Buddhist monks at Masoeyein Monastery. #May26Coup #AuspiceRefugees #WhatsHappeningInMyanmar https://t.co/CKHoNjJDAU
new message
RT @JusticeMyanmar: .@Total &amp; @Chevron made right 1st step suspending pipeline profits, finally acknowledging that MOGE is under military c…
new message
Every night, residents of WunTho Tsp, Sagaing Division, hold a praying service ceremony in which they chant "24 Patthana" to restore peace to Myanmar. #May26Coup #AuspiceRefugees

new message
RT @HanLay76523318: 🔴Happening in #Kayah state , Myanmar : Now, Military terrorists are attacking people from #kayah by using heavy
artille…
new message
Soros needs to be deported
new message
RT @12FightFuture21: (Sa-Taung)
(26/5/2021)
*Monks Praying Strike*
#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar

https://t.co/8h90G…
new message
RT @hannayuri_twt: The Myanmar military is reportedly abducting men in YinMee Village, PinLaung, Shan State. 
Military threatened,if people…
new message
(Sa-Taung)
(26/5/2021)
*Monks Praying Strike*
#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar

https://t.co/Nm4CV9jG26 https://t.co/5aeL0vEC7n
new message
RT @rinfujimatsu: ‼️⚠️#Myanmar military junta uses dead bodies as "tools of terror" @AP &amp; @hrcberkeley investigates👇🏼Bodies snatched &amp; drag…
new message
RT @MSrevolutor2021: Youths from #Bago, led a “Unity is Strength” Campaign to show that their solidarity with all ethnic people from #Myanm…
new message
RT @lisaeihsu: B

new message
RT @TotalPress: 🇲🇲 In light of the context in #Myanmar, and following a joint proposal by @Total &amp; @Chevron, shareholders of MGTC vote to s…
new message
RT @Myanmar_Now_Eng: In Taze, Sagaing, locals opposed to Myanmar’s military junta held a motorcycle rally this morning (May 26) and critici…
new message
RT @MyatMyintMoThin: The truth cannot be hidden
#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar
new message
RT @aapp_burma: Former political prisoner tortured to death by the junta. 

#WhatsHappeningInMyanmar
new message
The truth cannot be hidden
#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar https://t.co/07NpPk7E5J
new message
RT @Kate86613775: This is one of our success ✊

#WhatsHappeningInMyanmar 
#May26Coup
new message
This is one of our success ✊

#WhatsHappeningInMyanmar 
#May26Coup https://t.co/oBWYyvCaxQ
new message
Monks from Masoeyein monastery shared their prayers for the Myanmar people.

#May26Coup
#AuspiceRefugees
#WhatsHappeningInMyanmar https: